## Purpose of this notebook

- Examine similarities and differences between different raters' reliability report grades
- Calculate Cohen's kappa between pairs of raters
- Identify and print reports where raters strongly disagree (grade of 2 vs grade of 0)

## How to use this notebook

- Run each of the cells in order. Make sure you run Cell 01 first.
- Cells 02-03 get the proc_ord_id values unique to each report and the names of the persons who have graded reliability reports.
- Cells 04-07 can be modified to change which users you want to look at. 

In [1]:
# Cell 01: load libraries
from reliabilityLib import *
from google.cloud import bigquery # SQL table interface on Arcus
import pandas
import numpy
import matplotlib.pyplot as plt

In [2]:
# Cell 02: Get the list of proc_ord_id values used to identify the reliability reports
procIds = getReliabilityProcOrdIds()

In [3]:
# Cell 03: Get the distinct users
getDistinctUsersQuery = "select distinct grader_name as graders from lab.grader_table "
getDistinctUsersQuery += "where grade_category = 'Reliability' and grade != 999;"

client = bigquery.Client()
distinctUsers = list(client.query(getDistinctUsersQuery).to_dataframe()['graders'].values)
print(distinctUsers)

['Jisoo Kang', 'Megan M. Himes', 'Jenna Schabdach', 'Alesandra Gorgone', 'Sreya Subramanian']


In [5]:
# Cell 04:  Get the reliability reports for the users we want to evaluate (remember indexing in python starts at 0)
meganReports = getReportsForUser("Megan M. Himes", procIds)    
jennaReports = getReportsForUser("Jenna Schabdach", procIds)
allyReports = getReportsForUser("Alesandra Gorgone", procIds)

Megan M. Himes
(202, 2)
(151, 2)
Jenna Schabdach
(2152, 2)
(151, 2)
Alesandra Gorgone
(202, 2)
(151, 2)


In [6]:
# Cell 05: Compare the reliability reports for the users we want to evaluate
kappaMeganJenna = compareUserPairGrades(distinctUsers[1], meganReports, distinctUsers[2], jennaReports)
print("\n\n")
kappaMeganAlly = compareUserPairGrades(distinctUsers[1], meganReports, distinctUsers[3], allyReports)
print("\n\n")
kappaAllyJenna = compareUserPairGrades(distinctUsers[3], allyReports, distinctUsers[2], jennaReports)

Of the 55 reports Megan M. Himes gave a 0 , Jenna Schabdach gave
38 a grade of 0
14 a grade of 1
3 a grade of 2
Of the 32 reports Megan M. Himes gave a 1 , Jenna Schabdach gave
6 a grade of 0
24 a grade of 1
2 a grade of 2
Of the 64 reports Megan M. Himes gave a 2 , Jenna Schabdach gave
3 a grade of 0
16 a grade of 1
45 a grade of 2
 
With a Cohen's kappa of 0.565410779696494



Of the 55 reports Megan M. Himes gave a 0 , Alesandra Gorgone gave
49 a grade of 0
1 a grade of 1
5 a grade of 2
Of the 32 reports Megan M. Himes gave a 1 , Alesandra Gorgone gave
11 a grade of 0
13 a grade of 1
8 a grade of 2
Of the 64 reports Megan M. Himes gave a 2 , Alesandra Gorgone gave
0 a grade of 0
4 a grade of 1
60 a grade of 2
 
With a Cohen's kappa of 0.6927664351364624



Of the 60 reports Alesandra Gorgone gave a 0 , Jenna Schabdach gave
37 a grade of 0
21 a grade of 1
2 a grade of 2
Of the 18 reports Alesandra Gorgone gave a 1 , Jenna Schabdach gave
3 a grade of 0
15 a grade of 1
0 a grade of 2
O

In [10]:
# Cell 06: Identify the proc_ord_ids where each pair of users strongly disagree (one gave a 2, other gave a 0)
disagreementReportsMeganJenna = identifyDisagreementReports(meganReports, jennaReports)
print("Megan and Jenna disagree on", len(disagreementReportsMeganJenna), "reports\n")
disagreementReportsMeganAlly = identifyDisagreementReports(meganReports, allyReports)
print("Megan and Ally disagree on", len(disagreementReportsMeganAlly), "reports\n")
disagreementReportsAllyJenna = identifyDisagreementReports(allyReports, jennaReports)
print("Ally and Jenna disagree on", len(disagreementReportsAllyJenna), "reports\n")


Megan and Jenna disagree on 6 reports

Megan and Ally disagree on 5 reports

Ally and Jenna disagree on 9 reports



In [8]:
# Cell 07: 
# LAUREN - this is the cell where you can look at the disagreement reports for each pair of users
# User 1: Megan
# User 2: Jenna
printDisagreementReports(disagreementReportsMeganJenna, meganReports, jennaReports)